# **Data Wrangling**


## Hands on Lab


Importing Pandas:


In [1]:
import pandas as pd

Loading Dataset:


In [4]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m1_survey_data.csv")
df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
1,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,NaN,23.0,Man,No,Bisexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult
2,13,I am a developer by profession,Yes,Less than once a month but more than once per ...,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Somewhat more welcome now than last year,Tech articles written by other developers;Cour...,28.0,Man,No,Straight / Heterosexual,White or of European descent,Yes,Appropriate in length,Easy
3,16,I am a developer by profession,Yes,Never,The quality of OSS and closed source software ...,Employed full-time,United Kingdom,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,26.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult
4,17,I am a developer by profession,Yes,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,Employed full-time,Australia,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,29.0,Man,No,Straight / Heterosexual,Hispanic or Latino/Latina;Multiracial,No,Appropriate in length,Easy


## Finding duplicates


In [8]:
num_duplicate_rows = df.duplicated().sum()
print("Number of Duplicate Rows:", num_duplicate_rows)

Number of Duplicate Rows: 154


## Removing duplicates


In [9]:
df_no_duplicates = df.drop_duplicates()

print("Original DataFrame shape:", df.shape)
print("DataFrame shape without duplicates:", df_no_duplicates.shape)

Original DataFrame shape: (11552, 85)
DataFrame shape without duplicates: (11398, 85)


## Finding Missing values


In [10]:
missing_values_per_column = df.isnull().sum()

total_missing_values = df.isnull().sum().sum()

print("Missing Values per Column:")
print(missing_values_per_column)
print("\nTotal Number of Missing Values:", total_missing_values)

Missing Values per Column:
Respondent        0
MainBranch        0
Hobbyist          0
OpenSourcer       0
OpenSource       81
               ... 
Sexuality       547
Ethnicity       683
Dependents      144
SurveyLength     19
SurveyEase       14
Length: 85, dtype: int64

Total Number of Missing Values: 31381


Missing values in column **WorkLoc**:


In [11]:
missing_values_in_workloc = df['WorkLoc'].isnull().sum()

print("Number of Missing Values in 'WorkLoc' column:", missing_values_in_workloc)

Number of Missing Values in 'WorkLoc' column: 32


## Imputing missing values


Value Counts for **WorkLoc**:


In [12]:
df_no_duplicates['WorkLoc'].value_counts()

Office                                            6806
Home                                              3589
Other place, such as a coworking space or cafe     971
Name: WorkLoc, dtype: int64

Replacing missing values in **WorkLoc** with the most common (Office):


In [25]:
df_no_duplicates.loc[:,'WorkLoc'] = df_no_duplicates['WorkLoc'].fillna('Office')

/tmp/wsuser/ipykernel_288/3610416662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates.loc[:,'WorkLoc'] = df_no_duplicates['WorkLoc'].fillna('Office')


In [26]:
missing_values = df_no_duplicates['WorkLoc'].isnull().sum()

print("Number of Missing Values in 'WorkLoc' column:", missing_values)

Number of Missing Values in 'WorkLoc' column: 0


## Normalizing data


There are two columns in the dataset that talk about compensation.

One is "CompFreq". This column shows how often a developer is paid (Yearly, Monthly, Weekly).

The other is "CompTotal". This column talks about how much the developer is paid per Year, Month, or Week depending upon his/her "CompFreq". 

This makes it difficult to compare the total compensation of the developers.

I will create a new column called 'NormalizedAnnualCompensation' which contains the 'Annual Compensation' irrespective of the 'CompFreq'.

Once this column is ready, it makes comparison of salaries easy.


In [30]:
def normalize_compensation(row):
    if row['CompFreq'] == 'Yearly':
        return row['CompTotal']
    elif row['CompFreq'] == 'Monthly':
        return row['CompTotal'] * 12
    elif row['CompFreq'] == 'Weekly':
        return row['CompTotal'] * 52
    else:
        return None  
    
df_no_duplicates.loc[:, 'NormalizedAnnualCompensation'] = df_no_duplicates.apply(normalize_compensation, axis=1)

print("DataFrame with NormalizedAnnualCompensation column:")
print(df_no_duplicates[['CompFreq', 'CompTotal', 'NormalizedAnnualCompensation']])

DataFrame with NormalizedAnnualCompensation column:
      CompFreq  CompTotal  NormalizedAnnualCompensation
0       Yearly    61000.0                       61000.0
1       Yearly   138000.0                      138000.0
2       Yearly    90000.0                       90000.0
3      Monthly    29000.0                      348000.0
4       Yearly    90000.0                       90000.0
...        ...        ...                           ...
11547   Yearly   130000.0                      130000.0
11548   Yearly    74400.0                       74400.0
11549   Yearly   105000.0                      105000.0
11550   Yearly    80000.0                       80000.0
11551      NaN        NaN                           NaN

[11398 rows x 3 columns]


/tmp/wsuser/ipykernel_288/369010162.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates.loc[:, 'NormalizedAnnualCompensation'] = df_no_duplicates.apply(normalize_compensation, axis=1)
